# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, quniform, normal
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import os
import joblib
import shutil
import requests
import json


In [4]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-experiment'

experiment=Experiment(ws, experiment_name)


In [5]:


# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

Dataset is loaded in the train.py script!

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling(
    {
    "--max_depth": quniform(3, 7, 1),
    "--alpha": uniform(0.1, 10),
    "--learning_rate": uniform(0.05, 0.25),
    "--gamma": uniform(0.01, 5)
    }
)


# Create a SKLearn estimator for use with train.py
estimator = SKLearn(source_directory = '.',
              compute_target=compute_target,
              entry_script='train.py',
              pip_packages=["pyarrow>=0.12.0", "pyspark==3.0.1", "xgboost==0.90", "scikit-learn==0.24.0"])

primary_metric_name = "AUC_weighted"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     policy=None,
                                     max_total_runs=80,
                                     max_concurrent_runs=5,
                                     )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['scikit-learn']. We cannot guarantee image build will succeed.


In [7]:
hyperdrive_run = experiment.submit(hyperdrive_run_config, tag={"type": "hyperdrive_opt"})

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
hyperdrive_run.wait_for_completion(show_output=True)
assert (hyperdrive_run.get_status() == "Completed")
best_run = hyperdrive_run.get_best_run_by_primary_metric()

RunId: HD_1d6a9af1-ed68-4919-9233-c609e3ef8c29
Web View: https://ml.azure.com/experiments/hyperdrive-experiment/runs/HD_1d6a9af1-ed68-4919-9233-c609e3ef8c29?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-135176/workspaces/quick-starts-ws-135176

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-18T08:05:10.897338][API][INFO]Experiment created<END>\n""<START>[2021-01-18T08:05:11.966999][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n"<START>[2021-01-18T08:05:12.3342253Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-18T08:05:12.811709][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_1d6a9af1-ed68-4919-9233-c609e3ef8c29
Web View: https://ml.azure.com/experiments/hyperdrive-experiment/runs/HD_1d6a9af1-ed68-4919-9233-c609e3ef8c29?wsid=/subscri

In [10]:
print(best_run.get_metrics())

{'Max Depth:': 7, 'Alpha:': 1.3571583460407124, 'Learning rate:': 0.25, 'Gamma:': 0.01, 'AUC_weighted': 0.8688627769751112}


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
best_run.download_file("./outputs/wine-quality-model.pkl")
registered_model = best_run.register_model(model_name="wine-quality-model", model_path="./outputs/wine-quality-model.pkl")

myenv = Environment('my-environment')
conda_dep = CondaDependencies()

for conda_pkg in ["scikit-learn", "py-xgboost","numpy", "pip"]:
    conda_dep.add_conda_package(conda_pkg)

for pip_pkg in ["azureml-defaults","azureml-automl-core","azureml-automl-runtime","packaging", "inference-schema[numpy-support,pandas-support]"]:
    conda_dep.add_pip_package(pip_pkg)


with open("env.yml","w") as f:
    f.write(conda_dep.serialize_to_string())

myenv.python.conda_dependencies = conda_dep
service_name = 'wine-quality-classificator'

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.scoring_uri)
primary_key, _ = service.get_keys()

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://57fec84d-35ac-442a-9c51-8f913c8f8985.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [14]:
data={"data":
         [{ 
             "fixed acidity":7.90000,
             "volatile acidity":0.52000,
             "citric acid":0.26000, 
             "residual sugar":2.2000,
             "chlorides":0.07900,
             "free sulfur dioxide":14.0000,
             "total sulfur dioxide":38.0000,
             "density":0.99675,
             "pH":3.31000,
             "sulphates":0.62000,
             "alcohol":10.20000,
             "is_red_wine": 1
         },
         {
             "fixed acidity":6.80,
             "volatile acidity":0.26000,
             "citric acid":0.32000, 
             "residual sugar":5.2000,
             "chlorides":0.04300,
             "free sulfur dioxide":34.0000,
             "total sulfur dioxide":134.00000,
             "density":0.99374,
             "pH":3.18000,
             "sulphates":0.47000,
             "alcohol":10.40000,
             "is_red_wine": 0
         }]
     }
header = {'Content-Type': 'application/json'}
header["Authorization"] = f"Bearer {primary_key}"
json_data = json.dumps(data)

resp = requests.post(service.scoring_uri,json_data, headers=header)
print(resp.json())

{"result": [0, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
print(service.get_logs())
#service.delete()

2021-01-18T08:58:50,397423101+00:00 - iot-server/run 
2021-01-18T08:58:50,399979514+00:00 - nginx/run 
2021-01-18T08:58:50,399388811+00:00 - rsyslog/run 
2021-01-18T08:58:50,399207810+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c1774ef3adee538541c4a64455d2905d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC